In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama

In [15]:
message = "Powiedz mi cos o transformerach"
#response = ollama.chat(model='llama3.2', messages=[{"role":"user", "content":message}])
#response = ollama.chat(model='deepseek-r1:14b', messages=[{"role":"user", "content":message}])
response = ollama.chat(model='deepseek-r1:1.5b', messages=[{"role":"user", "content":message}])
print(response['message']['content'])

ResponseError: model is required (status code: 400)

In [5]:
# Niektóre strony internetowe wymagają użycia odpowiednich nagłówków podczas ich pobierania:
headers = {
"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

# Klasa reprezentująca stronę internetową, która zostanie pobeana do podsumowania

class Website:
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        # Użyjemy bibliotek BeautifulSoup do zbudowania biektu z treścią
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input", "link", "ul", "li", "a"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [11]:
web = Website("https://pl.wikipedia.org/wiki/Du%C5%BCy_model_j%C4%99zykowy")
print("Title: ", web.title)
print("text: ", web.text)

Title:  Duży model językowy – Wikipedia, wolna encyklopedia
text:  Menu główne
Menu główne
przypnij
ukryj
Nawigacja
Dla czytelników
Dla wikipedystów
Szukaj
Wygląd
Narzędzia osobiste
Spis treści
przypnij
ukryj
Przełącz stan spisu treści
Duży model językowy
56 języków
polski
Narzędzia
Narzędzia
przypnij
ukryj
Działania
Ogólne
Drukuj lub eksportuj
W innych projektach
Wygląd
przypnij
ukryj
Z Wikipedii, wolnej encyklopedii
Duży model językowy
(ang.
large language model
,
LLM
)
–
wyszkolony na dużej ilości danych umożliwiający generowanie tekstu oraz realizację zadań związanych z
. Modele
LLM
są szkolone w ramach
lub słabo nadzorowanego
z wykorzystaniem dużych ilości danych tekstowych. Proces ten jest intensywny obliczeniowo
. Duże modele językowe mogą być wykorzystywane do generowana tekstu poprzez wielokrotne przewidywanie następnego tokenu lub słowa, przez co zaliczane są do
.
Duże modele językowe są
. Największe i najbardziej zdolne modele językowe oparte są na architekturze
.
Przykładam

In [16]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown in Polish."

In [8]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a not long summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [9]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [13]:
messages_for(web)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown in Polish.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Duży model językowy – Wikipedia, wolna encyklopedia\nThe contents of this website is as follows; please provide a not long summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nMenu główne\nMenu główne\nprzypnij\nukryj\nNawigacja\nDla czytelników\nDla wikipedystów\nSzukaj\nWygląd\nNarzędzia osobiste\nSpis treści\nprzypnij\nukryj\nPrzełącz stan spisu treści\nDuży model językowy\n56 języków\npolski\nNarzędzia\nNarzędzia\nprzypnij\nukryj\nDziałania\nOgólne\nDrukuj lub eksportuj\nW innych projektach\nWygląd\nprzypnij\nukryj\nZ Wikipedii, wolnej encyklopedii\nDuży model językowy\n(ang.\nlarge language model\n,\nLLM\n)\n–\nwyszkolony na dużej ilości danych umożliwiający ge

In [17]:
MODEL='llama3.2'
def summarize(url):
    website = Website(url)
    response = ollama.chat(
        model=MODEL,
        messages=messages_for(website)
    )
    return response['message']['content']

In [19]:
#summarize("https://wp.pl")
summarize("https://ans-elblag.pl")

'**Podsumowanie strony**\n\nStrona **Akademii Nauk Stosowanych w Elblągu** to oficjalna strona instytucji edukacyjnej, która informuje o swojej działalności, wydarściach i niedawnych komunikatach prasowych.\n\n**Aktualności**\n\nOto najnowsze informacje zaktualnione w dniu 24 czerwca 2025:\n\n* Przygotowanie do studiów na kierunku **Przygotowanie Pedagogiczne**: trwa rekrutacja.\n* Konferencja Naukowa "Zachowania autodestrukcyjne dzieci i młodzieży. Współczesne wyzwania placówek oświatowych" odbyła się w przeszłości.\n* III Festiwal Wiedzy** z udziałem 170 uczniów z klas III z 6 Szkół Podstawowych z Elbląga.\n\n**Inne informacje**\n\nStrona ta wykorzystuje pliki cookies.'

In [20]:
def display_summary(url):
    summary = summarize (url)
    display (Markdown (summary))

In [21]:
display_summary("https://ans-elblag.pl")

**Strona główna Akademii Nauk Stosowanych w Elblągu**

* Akademia prowadzi studia na różne poziomy i oferuje kierunki naukowe, w tym podyplomowe.
* Aktualnie trwa rekrutacja na studia podyplomowe na kierunku Przygotowanie Pedagogiczne.
* W przeszłości odbyły się wydarzenia takie jak Konferencja Naukowa "Zachowania autodestrukcyjne dzieci i młodzieży. Współczesne wyzwania placówek oświatowych" oraz III Festiwal Wiedzy, w którym udział wzięło 170 uczniów.
* Akademia ma certyfikat ISO 9001, co wskazuje na jej niezawodność i jakość.